### testing S2 imagery composites
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

So far 'max' composite from the S2A product is not very good (too much cloud). If want to use 'max' composites, might need to try 95th %ile rather than max, or possibly a more aggressive cloud mask. use of the 'S2A cloudless' product could be a good option for this: https://medium.com/google-earth/more-accurate-and-flexible-cloud-masking-for-sentinel-2-images-766897a9ba5f

**To do:** determine which bands are worth keeping for modelling - prob dont wand B1 (aerosol) or B9 (water vapour) at least. Not sure about red edge bands ('B5','B6', 'B7', 'B8A') as they are 20m, but at least one of these is probably useful for veg. Current selection: ['B2', 'B3', 'B4', 'B5','B6', 'B7', 'B8', 'B11', 'B12']

Name	Units	Min	Max	Scale	Pixel Size	Wavelength	Description


B1				0.0001	60 meters	443.9nm (S2A) / 442.3nm (S2B)	Aerosols

B2				0.0001	10 meters	496.6nm (S2A) / 492.1nm (S2B)	Blue

B3				0.0001	10 meters	560nm (S2A) / 559nm (S2B)	Green

B4				0.0001	10 meters	664.5nm (S2A) / 665nm (S2B)	Red

B5				0.0001	20 meters	703.9nm (S2A) / 703.8nm (S2B)	Red Edge 1

B6				0.0001	20 meters	740.2nm (S2A) / 739.1nm (S2B)	Red Edge 2

B7				0.0001	20 meters	782.5nm (S2A) / 779.7nm (S2B)	Red Edge 3

B8				0.0001	10 meters	835.1nm (S2A) / 833nm (S2B)	NIR

B8A				0.0001	20 meters	864.8nm (S2A) / 864nm (S2B)	Red Edge 4

B9				0.0001	60 meters	945nm (S2A) / 943.2nm (S2B)	Water vapor

B11				0.0001	20 meters	1613.7nm (S2A) / 1610.4nm (S2B)	SWIR 1

B12				0.0001	20 meters	2202.4nm (S2A) / 2185.7nm (S2B)	SWIR 2

In [1]:
import ee
from geemap import geemap

In [12]:
def maskS2clouds(image):
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

# rename bands
#def renameS2bands(image):
#    return image.rename('B', 'G', 'R', 'NIR', 'SWIR', 'SWIR2', 'QA60')
def renameS2bands(image):
    return image.rename('B2', 'B3', 'B4', 'B8', 'B11', 'B12')


rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'], 
}

In [3]:
fp_train_ext1 = "/home/markdj/Dropbox/artio/polesia/val/Vegetation_extent_rough.shp"
fp_roi = "/home/markdj/Dropbox/artio/polesia/val/roi_rough.shp"

In [4]:
roi_test = geemap.shp_to_ee(fp_roi)
roi_train = geemap.shp_to_ee(fp_train_ext1)

In [76]:
start_date_list=['2019-05-01']

### mask cloud using QA60 band

In [28]:
start_date = start_date_list[0]
# load Sentinel-2 TOA reflectance data.
s2collection = ee.ImageCollection('COPERNICUS/S2_SR') \
              .filterDate(start_date, ee.Date(start_date).advance(1, 'month')) \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
              .filterBounds(roi_train) \
              .select('B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60') \
              .map(maskS2clouds) #\
              #.map(renameS2bands)

#Reduce the collection, and clip to train roi
s2_median = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').median().clip(roi_train.geometry())
s2_min = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').min().clip(roi_train.geometry())
s2_max = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').max().clip(roi_train.geometry())
s2_75 = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').reduce(ee.Reducer.percentile([75])).clip(roi_train.geometry())
#Reducer.percentile has an outputNames arg, but don't think option to give names matching the input names 
s2_75 = renameS2bands(s2_75)

# data stack
#s2_multiple = s2_median.addBands(s2_min).addBands(s2_max)

In [42]:
Map = geemap.Map(center=(51.85, 27.8), zoom=9)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
#Map.add_basemap('SATELLITE')

Map.addLayer(s2_min, rgbVis, 's2min')
Map.addLayer(s2_median, rgbVis, 's2med')
Map.addLayer(s2_max, rgbVis, 's2max')
Map.addLayer(s2_75, rgbVis, 's2p75')
#Map.addLayer(roi_train, {}, 'ROI_train')
#Map.addLayer(roi_test, {}, 'ROI_test')

Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

Cloud masking in the above is not very good... cloud visible in the median and 75 %ile images. The below compares the existing approach to a slightly different masking function from here: https://courses.spatialthoughts.com/end-to-end-gee.html

In [72]:
# Get the Level-2A Surface Reflectance image
imageSR = ee.Image('COPERNICUS/S2_SR/20190703T050701_20190703T052312_T43PGP')

Map = geemap.Map(center=(51.85, 27.8), zoom=9)
Map.centerObject(imageSR)

# // Function to remove cloud and snow pixels from Sentinel-2 SR image
# function maskCloudAndShadowsSR(image) {
#   var cloudProb = image.select('MSK_CLDPRB');
#   var snowProb = image.select('MSK_SNWPRB');
#   var cloud = cloudProb.lt(5);
#   var snow = snowProb.lt(5);
#   var scl = image.select('SCL'); 
#   var shadow = scl.eq(3); // 3 = cloud shadow
#   var cirrus = scl.eq(10); // 10 = cirrus
#   // Cloud probability less than 5% or cloud shadow classification
#   var mask = (cloud.and(snow)).and(cirrus.neq(1)).and(shadow.neq(1));
#   return image.updateMask(mask);
# }

## Function to remove cloud and snow pixels from Sentinel-2 SR image
## MdJ:i think if you want to use a function with 'map', you can't pass args?
def maskCloudAndShadowsSR(image):
    cloudProb = image.select('MSK_CLDPRB')
    snowProb = image.select('MSK_SNWPRB')
    cloud = cloudProb.lt(2)
    snow = snowProb.lt(2)
    scl = image.select('SCL') 
    shadow = scl.eq(3) # 3 = cloud shadow
    cirrus = scl.eq(10) # 10 = cirrus
    # Cloud probability less than 5% or cloud shadow classification
    #mask = (cloud.and(snow)).and(cirrus.neq(1)).and(shadow.neq(1))
    mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
    return image.updateMask(mask)

imageSRmasked1 = maskS2clouds(imageSR)
imageSRmasked2 = maskCloudAndShadowsSR(imageSR)

Map.addLayer(imageSR, rgbVis, 'SR Image')
Map.addLayer(imageSRmasked1, rgbVis, 'SR Image masked1')
Map.addLayer(imageSRmasked2, rgbVis, 'SR Image masked2')
Map

Map(center=[12.658204357875261, 76.8410807013455], controls=(WidgetControl(options=['position', 'transparent_b…

new approach looks better. let's try to apply it to the polesia composites...

In [82]:
start_date = start_date_list[0]
# load Sentinel-2 TOA reflectance data.
s2collection = ee.ImageCollection('COPERNICUS/S2_SR') \
              .filterDate(start_date, ee.Date(start_date).advance(3, 'month')) \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
              .filterBounds(roi_train) \
              .map(maskCloudAndShadowsSR) #\
              #.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60') \
              #.map(renameS2bands)

#Reduce the collection, and clip to train roi
s2_median = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').median().clip(roi_train.geometry())
s2_min = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').min().clip(roi_train.geometry())
s2_max = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').max().clip(roi_train.geometry())
s2_75 = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').reduce(ee.Reducer.percentile([75])).clip(roi_train.geometry())
#Reducer.percentile has an outputNames arg, but don't think option to give names matching the input names 
s2_75 = renameS2bands(s2_75)

# data stack
#s2_multiple = s2_median.addBands(s2_min).addBands(s2_max)

Map = geemap.Map(center=(51.85, 27.8), zoom=9)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
#Map.add_basemap('SATELLITE')

Map.addLayer(s2_min, rgbVis, 's2min')
Map.addLayer(s2_median, rgbVis, 's2med')
Map.addLayer(s2_max, rgbVis, 's2max')
#Map.addLayer(s2_75, rgbVis, 's2p75')

#Map.addLayer(roi_train, {}, 'ROI_train')
#Map.addLayer(roi_test, {}, 'ROI_test')

Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

while the above is much better than the first attempt, there are still a bunch of cloud edge artifacts in the max and 75%ile outputs even when cloud probability thresh is at it's strictest (<2). thinking about it, it makes sense that unless you have really good masking, you will always have contamination of min/max composits by cloud or cloud shadow. maybe better just using median to avoid these issues?

median still contains some cloud not captured by the current makss, but seems to imporve when we aggregate over multiple months. so maybe doing a 3 month summer and winter composite is the best way forwards

Also - try the cloudless product? https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

### S2A cloudless product
most of this code is from here:

https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

In [86]:
AOI = roi_train
START_DATE = '2019-06-01'
END_DATE = '2019-06-30'
CLOUD_FILTER = 60       # filter scenes with cloud prop > X from collection
CLD_PRB_THRESH = 40     # cloud prob > thresh = cloud
NIR_DRK_THRESH = 0.15   # percentile used to determine dark non-water areas of scene using Band 8
CLD_PRJ_DIST = 2        # max distnce (*10) for cloud shadows to be projected (units??)
BUFFER = 100            # applied to mask cloud edges

In [87]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [95]:
bands = ['B2', 'B3', 'B4', 'B5','B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

# composites
s2cldless_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .select(bands) 
                             .median()
                             .clip(roi_train.geometry()))
s2cldless_min = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .select(bands)                  
                             .min()
                             .clip(roi_train.geometry()))
s2cldless_max = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .select(bands)                  
                             .max()
                             .clip(roi_train.geometry()))

In [163]:
print(s2cldless_median)

ee.Image({
  "type": "Invocation",
  "arguments": {
    "input": {
      "type": "Invocation",
      "arguments": {
        "collection": {
          "type": "Invocation",
          "arguments": {
            "collection": {
              "type": "Invocation",
              "arguments": {
                "collection": {
                  "type": "Invocation",
                  "arguments": {
                    "collection": {
                      "type": "Invocation",
                      "arguments": {
                        "join": {
                          "type": "Invocation",
                          "arguments": {
                            "matchKey": "s2cloudless"
                          },
                          "functionName": "Join.saveFirst"
                        },
                        "primary": {
                          "type": "Invocation",
                          "arguments": {
                            "collection": {
                          

In [94]:
Map = geemap.Map(center=(51.85, 27.8), zoom=9)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
#Map.add_basemap('SATELLITE')

Map.addLayer(s2_min, rgbVis, 's2min')
Map.addLayer(s2cldless_median, rgbVis, 's2s2cldless_medianmin')

#Map.addLayer(s2_median, rgbVis, 's2_median')
#Map.addLayer(s2cldless_median, rgbVis, 's2cldless_median')

#Map.addLayer(s2_max, rgbVis, 's2_max')
#Map.addLayer(s2cldless_max, rgbVis, 's2cldless_max')

Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

while far from perfect, the new S2cloudless based masking gives more visually pleasing composites - little visible residual cloud remains, and in other areas it leaves masked areas (gaps). 

Also can see in area of agriculture to the West of the map there are 'bubbles' of green crop in senesced crop indicating compositing in can result in artifacts even over short (month) long periods. However, this same area also looks a bit shit in the basic cloud mask version, but is 'patchy' rather than 'bubbly'


### test stack generation
shouldnt really do gee with a 'for' loop...

In [169]:
AOI = roi_train
# START_DATE = '2019-06-01'
START_DATE_LIST=['2019-05-01', '2019-06-01']
# END_DATE = '2019-06-30'
CLOUD_FILTER = 60       # filter scenes with cloud prop > X from collection
CLD_PRB_THRESH = 40     # cloud prob > thresh = cloud
NIR_DRK_THRESH = 0.15   # percentile used to determine dark non-water areas of scene using Band 8
CLD_PRJ_DIST = 2        # max distnce (*10) for cloud shadows to be projected (units??)
BUFFER = 100            # applied to mask cloud edges
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']


for i, START_DATE in enumerate(START_DATE_LIST):
    mnth=i+1
    print(START_DATE)
    END_DATE = ee.Date(START_DATE).advance(1, 'month')
    
    #collection
    s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
    
    # composites
    s2cldless_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                 .map(apply_cld_shdw_mask)
                                 .select(bands) 
                                 .median()
                                 .clip(roi_train.geometry())
                                 .rename(f'{mnth}_B2', f'{mnth}_B3', f'{mnth}_B4', f'{mnth}_B5', 
                                               f'{mnth}_B6', f'{mnth}_B7', f'{mnth}_B8', f'{mnth}_B8A',
                                               f'{mnth}_B11', f'{mnth}_B12'))    
    if i == 0:
        median_stack = s2cldless_median
    else:
        median_stack = median_stack.addBands(s2cldless_median)
       

2019-05-01
2019-06-01


In [170]:
median_stack.getInfo()

{'type': 'Image',
 'bands': [{'id': '1_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '1_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '1_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '1_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 2],
   'origin': [27, 51],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '1_B6',
   'data_type': {'type': 'PixelType',
    'preci

In [172]:
vis_1 = {'min': 0.0,'max': 3000,'bands': ['1_B4', '1_B3', '1_B2']}
vis_2 = {'min': 0.0,'max': 3000,'bands': ['2_B4', '2_B3', '2_B2']}

Map = geemap.Map(center=(51.85, 27.8), zoom=9)
Map.addLayer(median_stack, vis_1, 'month1')
Map.addLayer(median_stack, vis_2, 'month2')
Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

### Alternate stacking
trying to use a map() to iterate over months rather than a loop

In [158]:
AOI = roi_train
# START_DATE = '2019-06-01'
# START_DATE_LIST=['2019-05-01', '2019-06-01']
# END_DATE = '2019-06-30'
CLOUD_FILTER = 60       # filter scenes with cloud prop > X from collection
CLD_PRB_THRESH = 40     # cloud prob > thresh = cloud
NIR_DRK_THRESH = 0.15   # percentile used to determine dark non-water areas of scene using Band 8
CLD_PRJ_DIST = 2        # max distnce (*10) for cloud shadows to be projected (units??)
BUFFER = 100            # applied to mask cloud edges
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']


start_date = ee.Date('2019-01-01')
end_date = ee.Date('2019-12-31')
nMonths = ee.Number(end_date.difference(start_date,'month')).round()

def filtr(n):
    ini = start_date.advance(n,'month')
    end = ini.advance(1,'month')
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(roi_train)
        .filterDate(ini, end))
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(roi_train)
        .filterDate(ini, end))
    s2_sr_cld_col = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    })) 
    
    s2cldless_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                 .map(apply_cld_shdw_mask)
                                 .select(bands) 
                                 .median()
                                 .set('month',n))
                                 #.clip(roi_train.geometry()))
    return s2cldless_median
    
    
result2 = ee.List.sequence(0,nMonths.subtract(1)).map(filtr)    
myCollection = ee.ImageCollection.fromImages(result2)

now i think i've created a collection of median composits (1 per month), but struggling to access/map them!

In [149]:
image_ids = myCollection.aggregate_array('system:index').getInfo()
image_ids

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

In [152]:
image = ee.Image(myCollection.filter(ee.Filter.eq('system:index', 0)).first())
image

In [159]:
# trying to extract things for plotting and verification...
listOfImages = ee.Image(myCollection.toList(myCollection.size()));
firstImage = ee.Image(listOfImages.get(0));
secondImage = ee.Image(listOfImages.get(1));
#lastImage = ee.Image(listOfImages.get(listOfImages.length().subtract(1)));
#Now you can use ee.Image functions 
B2 = firstImage.select("B2")


In [161]:
Map = geemap.Map(center=(51.85, 27.8), zoom=9)
Map.addLayer(firstImage, rgbVis, 's2min')
Map

EEException: Element.get, argument 'property': Invalid type.
Expected type: String.
Actual type: Integer.
Actual value: 0